# Hoja de Trabajo 2

Gabriel García - 21352        
Luis Montenegro - 21699

# Ejercicio 1 - Experimentación Practica

En esta actividad, implementará y comparará diferentes funciones de pérdida y técnicas de regularización utilizando PyTorch. Utilizará el conjunto de datos de Iris para una tarea de clasificación y una arquitectura básica de red neuronal de feedforward. El objetivo es observar cómo las diferentes opciones impactan la convergencia y el rendimiento del modelo.

In [7]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
import numpy as np

# Task 1 - Preparación del conjunto de datos

Cargue el conjunto de datos de Iris utilizando bibliotecas como sklearn.datasets. Luego, divida el conjunto de datos en conjuntos de entrenamiento y validación

In [8]:
# Cargar el dataset de iris
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [9]:
# Escalar los datos
scaler = StandardScaler()
X = scaler.fit_transform(x)

In [10]:
# Dividir el conjunto de datos en conjuntos de entrenamiento y validación (80% entrenamiento, 20% validación)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [11]:
# Convertir los datos a tensores de PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

In [13]:
# Crear conjuntos de datos y cargadores de datos
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [14]:
# Imprimir las formas de los conjuntos de datos para verificación
print(f"Tamaño del conjunto de entrenamiento: {len(train_loader.dataset)} muestras")
print(f"Tamaño del conjunto de validación: {len(val_loader.dataset)} muestras")

Tamaño del conjunto de entrenamiento: 120 muestras
Tamaño del conjunto de validación: 30 muestras
